In [8]:
# Import necessary libraries and modules
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, GroupChat, GroupChatManager
import replicate
from datetime import datetime
import requests
from PIL import Image, ImageDraw, ImageFont
import io
from dotenv import load_dotenv
import os

load_dotenv()

# Set up OpenAI API key
api_key = os.getenv('OPENAI_API_KEY')

# Load the configuration list from a JSON file or environment variable
config_list = config_list = [
    {
        "model": "gpt-4o",
        "api_key": api_key,
        "tags": ["gpt-4o", "tool"]
    }
]

# Define a configuration for the Language Model (LLM) and AutoGen
llm_config = {
    "config_list": config_list,
    "timeout": 120
}

# Define a function to review an image
def img_review(image_path, prompt):
    # Simulating image review with AutoGen
    return f"Review for '{prompt}': Image looks great! 8/10."

# Define a function to generate an image from text using Pillow
def text_to_image_generation(prompt):
    exec_globals = {}
    # Generate image using Pillow based on the prompt
    exec(prompt, {"Image": Image, "ImageDraw": ImageDraw, "ImageFont": ImageFont, "io": io}, exec_globals)
    image = exec_globals.get('image')
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    draw.text((10, 10), prompt, fill='black', font=font)

    # Save the image with a filename based on prompt and current time
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    shortened_prompt = prompt[:50]
    filename = f"image/{shortened_prompt}_{current_time}.png"
    image.save(filename)

    return f"Image saved as '{filename}'"

# Define configuration for the assistant agents
llm_config_assistants = {
    "functions": [
        {
            "name": "text_to_image_generation",
            "description": "Utilize the most recent AI model to create a python code that uses pillow to create the image using a given prompt and provide the file path to the generated image.",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {
                        "type": "string",
                        "description": "Create a Python script to generate an image for the prompt. Only respond a python code and Only respond with code as plain text without code block syntax around it.",
                    }
                },
                "required": ["prompt"],
            },
        },
        {
            "name": "image_review",
            "description": "Examine and assess the image created by AI according to the initial prompt, offering feedback and recommendations for enhancement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {
                        "type": "string",
                        "description": "The original input text that served as the prompt for generating the image.",
                    },
                    "image_path": {
                        "type": "string",
                        "description": "The complete file path for the image, including both the directory path and the file extension.",
                    }
                },
                "required": ["prompt", "image_path"],
            },
        },
    ],
    "config_list": config_list,
    "timeout": 120
}

# Create assistant agents with specific roles
img_gen_assistant = AssistantAgent(
    name="text_to_img_prompt_expert",
    system_message="As an expert in text-to-image AI models, you will utilize the 'text_to_image_generation' function to create an image based on the given prompt and iterate on the prompt, incorporating feedback until it achieves a perfect rating of 10/10.",
    llm_config=llm_config_assistants,
    function_map={
        "image_review": img_review,
        "text_to_image_generation": text_to_image_generation
    }
)

img_critic_assistant = AssistantAgent(
    name="img_critic",
    system_message="In the role of an AI image critic, your task is to employ the 'image_review' function to evaluate the image generated by the 'text_to_img_prompt_expert' using the original prompt. You will then offer feedback on how to enhance the prompt for better image generation.",
    llm_config=llm_config_assistants,
    function_map={
        "image_review": img_review,
        "text_to_image_generation": text_to_image_generation
    }
)

# Create a user proxy agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

# Create a group chat with the defined agents
groupchat = GroupChat(
    agents=[user_proxy, img_gen_assistant, img_critic_assistant],
    messages=[],  # The initial messages in the chat
    max_round=10  # Maximum rounds of conversation
)

# Create a group chat manager
manager = GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config
)

# Start the conversation by having the user proxy initiate the chat
user_proxy.initiate_chat(
    manager, message=
"""
from PIL import Image, ImageDraw
# Creating a new image (size: 500x500, color: blue - to represent the sea)
img = Image.new('RGB', (500, 500), color = (0, 75, 150))

d = ImageDraw.Draw(img)

# Drawing a rectangle to represent the beach
d.rectangle([0,400,500,500], fill=(194, 178, 128))

img.show()
"""
)


user_proxy (to chat_manager):


from PIL import Image, ImageDraw
# Creating a new image (size: 500x500, color: blue - to represent the sea)
img = Image.new('RGB', (500, 500), color = (0, 75, 150))

d = ImageDraw.Draw(img)

# Drawing a rectangle to represent the beach
d.rectangle([0,400,500,500], fill=(194, 178, 128))

img.show()


--------------------------------------------------------------------------------



Next speaker: text_to_img_prompt_expert

text_to_img_prompt_expert (to chat_manager):

***** Suggested function call: text_to_image_generation *****
Arguments: 
{
  "prompt": "A serene beach scene with a blue sea, a sandy shore, and a calm atmosphere"
}
*************************************************************

--------------------------------------------------------------------------------

Next speaker: img_critic


>>>>>>>> EXECUTING FUNCTION text_to_image_generation...
img_critic (to chat_manager):

***** Response from calling function (text_to_image_generation) *****
Error: invalid syntax (<string>, line 1)
*********************************************************************

--------------------------------------------------------------------------------

Next speaker: text_to_img_prompt_expert

text_to_img_prompt_expert (to chat_manager):

It seems like there was some issue with the previous code. Let me generate the correct Python code to create the specified beach scene.

ChatResult(chat_id=None, chat_history=[{'content': "\nfrom PIL import Image, ImageDraw\n# Creating a new image (size: 500x500, color: blue - to represent the sea)\nimg = Image.new('RGB', (500, 500), color = (0, 75, 150))\n\nd = ImageDraw.Draw(img)\n\n# Drawing a rectangle to represent the beach\nd.rectangle([0,400,500,500], fill=(194, 178, 128))\n\nimg.show()\n", 'role': 'assistant'}, {'content': '', 'function_call': {'arguments': '{\n  "prompt": "A serene beach scene with a blue sea, a sandy shore, and a calm atmosphere"\n}', 'name': 'text_to_image_generation'}, 'name': 'text_to_img_prompt_expert', 'role': 'assistant'}, {'content': 'Error: invalid syntax (<string>, line 1)', 'name': 'text_to_image_generation', 'role': 'function'}, {'content': 'It seems like there was some issue with the previous code. Let me generate the correct Python code to create the specified beach scene.\n\n```python\nfrom PIL import Image, ImageDraw\n\n# Creating a new image (size: 500x500, color: blue - to rep